In [ ]:
# Importing necessary lebraries
import numpy as np
import pandas as pd
import os
import sys
import warnings
import logging
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
sys.path.append(os.path.join(os.path.abspath('..')))
# Import modules
from src import data_loading as dl
from src.fetcher import WorldBankDataFetcher
sys.path.append(os.path.abspath("../"))

# Now import your scripts
#from scripts.eda import check_missing_values, check_duplicates, check_outliers,check_stationarity, save_cleaned_data

from scripts.visualizer import plot_historical_prices,plot_rolling_statistics,plot_price_trend_over_years,plot_with_events,plot_time_series_decomposition,plot_smoothed_probabilities
from scripts.analyzer import analyze_events,preprocess_data,fit_markov_switching_model,process_data
from scripts.oil_price_analysis import analyze_indicators
from scripts.AdaptingModel import PricePredictor

In [ ]:
logger = logging.getLogger('EDA Logger')
logger.setLevel(logging.DEBUG)
# load the data
data = dl.load_data("BrentOilPrices.csv")
print("head of the BrentOilPrices ")
data.head()

In [ ]:
data["Date"] = pd.to_datetime(data["Date"])
data = data.set_index("Date")
# Plot historical prices
plot_historical_prices(data)

In [ ]:
plot_with_events(data)


In [ ]:
# Call the function
analyze_events(data)

In [ ]:
# Preprocess the data
diff_data = process_data(data)

# Fit the model
results = fit_markov_switching_model(diff_data)
print(results.summary())

In [ ]:
# Plot the smoothed probabilities
plot_smoothed_probabilities(results, diff_data)

In [ ]:
!pip install pandas numpy matplotlib scikit-learn tensorflow


In [ ]:
import sys
sys.modules.pop('scripts.analyzer')
from scripts.analyzer import build_lstm_model, train_model, evaluate_model, preprocess_data
# Preprocess the data for LSTM
X_train, y_train, X_test, y_test, scaler, train_size, time_step = preprocess_data(data)
model = build_lstm_model(input_shape=(X_train.shape[1], 1))
train_model(model, X_train, y_train, epochs=10, batch_size=32)

In [ ]:
predictions, mse, rmse, mae = evaluate_model(model, X_test, y_test, scaler)


In [ ]:
!pip install wbdata


In [ ]:
# Import the WorldBankDataFetcher class


# Define your indicators
indicators = {
    'NY.GDP.MKTP.CD': {'name': 'GDP', 'country': 'WLD'},
    'FP.CPI.TOTL.ZG': {'name': 'CPI', 'country': 'WLD'},
    'SL.UEM.TOTL.ZS': {'name': 'Unemployment_Rate', 'country': 'WLD'},
    'PA.NUS.FCRF': {'name': 'Exchange_Rate', 'country': 'EMU'}
}

# Initialize the fetcher
fetcher = WorldBankDataFetcher('1987-05-20', '2022-11-14')

# Fetch, process, and save data for each indicator
for indicator_code, info in indicators.items():
    raw_data = fetcher.fetch_indicator_data(indicator_code, info['name'], info['country'])
    processed_data = fetcher.process_data(raw_data, info['name'])
    if not processed_data.empty:
        fetcher.save_data(processed_data, info['name'])
processed_data.to_csv("data/CPI_cleaned_data_daily.csv", index=False)

# Now you can load and use the saved data in your notebook

import pandas as pd

gdp_data = pd.read_csv('data/GDP_cleaned_data_daily.csv')
cpi_data = pd.read_csv('data/CPI_cleaned_data_daily.csv')
unemployment_data = pd.read_csv('data/Unemployment_Rate_cleaned_data_daily.csv')
exchange_rate_data = pd.read_csv('data/Exchange_Rate_cleaned_data_daily.csv')

In [ ]:
sys.path.append(os.path.abspath("../"))



# Load the data
try:
    gdp_data_daily = pd.read_csv("data/GDP_cleaned_data_daily.csv")
    gdp_data_daily['Date'] = pd.to_datetime(gdp_data_daily['Date'])

    cpi_data_daily = pd.read_csv("data/CPI_cleaned_data_daily.csv")
    cpi_data_daily['Date'] = pd.to_datetime(cpi_data_daily['Date'])

    unemployment_data_daily = pd.read_csv("data/Unemployment_Rate_cleaned_data_daily.csv")
    unemployment_data_daily['Date'] = pd.to_datetime(unemployment_data_daily['Date'])

    exchange_rate_data_daily = pd.read_csv("data/Exchange_Rate_cleaned_data_daily.csv")
    exchange_rate_data_daily['Date'] = pd.to_datetime(exchange_rate_data_daily['Date'])

    oil_data_daily = pd.read_csv("data/BrentOilPrices.csv")
    oil_data_daily['Date'] = pd.to_datetime(oil_data_daily['Date'])
    oil_data_daily.set_index('Date', inplace=True)

    print("Data loaded successfully!")

except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please ensure all required CSV files are in the correct directory.")
    raise

# Run the analysis
if 'gdp_data_daily' in locals() and 'cpi_data_daily' in locals() and \
   'unemployment_data_daily' in locals() and 'exchange_rate_data_daily' in locals() and \
   'oil_data_daily' in locals():

    analyze_indicators(
        gdp_data_daily,
        cpi_data_daily,
        unemployment_data_daily,
        exchange_rate_data_daily,
        oil_data_daily
    )
else:
    print("Some required data files could not be loaded. Please check the data paths and file names.")

In [ ]:
# Initialize the predictor
predictor = PricePredictor()

# Load data
if predictor.load_data():
    # Train LSTM model
    predictor.train_lstm()